In [1]:
import numpy as np
import cv2
from scipy.optimize import least_squares

# Function to project 3D points to 2D using camera matrix
def project(points_3d, camera_params):
    points_proj = []
    for point in points_3d:
        point = np.append(point, 1) # Convert to homogeneous
        point_proj = camera_params @ point
        point_proj /= point_proj[2] # Normalize by depth
        points_proj.append(point_proj[:2])
    return np.array(points_proj)

# Residual function for bundle adjustment
def residuals(params, n_cameras, n_points, camera_indices, point_indices, points_2d):
    camera_params = params[:n_cameras * 12].reshape((n_cameras, 3, 4)) # Assuming P matrix for each camera
    points_3d = params[n_cameras * 12:].reshape((n_points, 3))
    points_proj = project(points_3d[point_indices], camera_params[camera_indices])
    return (points_proj - points_2d).ravel()

# Simulated data
n_cameras = 2
n_points = 10
points_3d = np.random.rand(n_points, 3) # Random 3D points
camera_params = np.array([np.eye(3, 4) for _ in range(n_cameras)]) # Identity cameras
camera_indices = np.random.randint(0, n_cameras, n_points)
point_indices = np.arange(n_points)
points_2d = project(points_3d, camera_params[camera_indices])
# Bundle adjustment using least squares
initial_params = np.hstack((camera_params.ravel(), points_3d.ravel()))
result = least_squares(residuals, initial_params, verbose=2, x_scale='jac', ftol=1e-4,
 args=(n_cameras, n_points, camera_indices, point_indices,
points_2d))
# Optimized camera parameters and 3D points
optimized_camera_params = result.x[:n_cameras * 12].reshape((n_cameras, 3, 4))
optimized_points_3d = result.x[n_cameras * 12:].reshape((n_points, 3))
print("Optimized camera parameters:\n", optimized_camera_params)
print("Optimized 3D points:\n", optimized_points_3d)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         0.0000e+00                                    0.00e+00    
`gtol` termination condition is satisfied.
Function evaluations 1, initial cost 0.0000e+00, final cost 0.0000e+00, first-order optimality 0.00e+00.
Optimized camera parameters:
 [[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]]
Optimized 3D points:
 [[0.53228892 0.25415994 0.90690294]
 [0.4866271  0.02114636 0.81909108]
 [0.75688752 0.36173023 0.75977829]
 [0.27770409 0.17993612 0.41967442]
 [0.28856247 0.77487455 0.97050187]
 [0.71232213 0.19286545 0.1363532 ]
 [0.05254343 0.67040688 0.23472403]
 [0.81176935 0.62852421 0.24731894]
 [0.15959266 0.60895955 0.21032366]
 [0.53265377 0.33855335 0.67653526]]
